### Lecture 4: Homework

Today we gonna learn how to choose between ML models, based on data type. Your task would be to predict **the edibility of a mushroom** based on sample descriptions (binary classification problem)

The **tricky part here is that 95% of the features are of categorical type.**
<br>That's the one where we would **(usually)  prefer tree-based algorithms over linear methods**

Although this dataset was originally contributed to the UCI Machine Learning repository nearly 30 years ago, mushroom hunting (otherwise known as "shrooming") is enjoying new peaks in popularity. Learn which features spell certain death and which are most palatable in this dataset of mushroom characteristics. And how certain can your model be?

This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom drawn from The Audubon Society Field Guide to North American Mushrooms (1981). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like "leaflets three, let it be'' for Poisonous Oak and Ivy.

More information can be found [here](https://www.kaggle.com/uciml/mushroom-classification/data)

In [2]:
# library import
import pandas as pd
import numpy as np
from os.path import join as pjoin
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

# preprocessing / validation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
# ML models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# metrics
from sklearn.metrics import classification_report, f1_score

In [3]:
# read data
df_train = pd.read_csv( '4-mushrooms-train.csv',engine='c')
df_test = pd.read_csv( '4-mushrooms-test.csv',engine='c')
print(df_train.shape, df_test.shape)

(6499, 23) (1625, 22)


In [4]:
# let's see what data looks like
df_train.head()

,target,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,0,convex,scaly,brown,bruises,pungent,free,close,narrow,white,enlarging,equal,smooth,smooth,white,white,partial,white,one,pendant,brown,scattered,urban
1,1,flat,fibrous,gray,bruises,none,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,white,partial,white,one,pendant,brown,several,woods
2,0,flat,smooth,brown,no,none,attached,close,broad,orange,enlarging,missing,smooth,smooth,orange,orange,partial,orange,one,pendant,brown,several,leaves
3,1,convex,fibrous,gray,bruises,none,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,white,partial,white,one,pendant,black,solitary,woods
4,0,knobbed,smooth,brown,no,foul,free,close,narrow,buff,tapering,missing,silky,smooth,pink,pink,partial,white,one,evanescent,white,several,paths


In [5]:
# for convenient calculations, let us merge train with test
df = pd.concat([df_train, df_test], axis=0)
# add column for filtering train/test
df['is_train'] = True
df.loc[df.target.isnull(), 'is_train'] = False
# check shapes
print(df.shape)
# check labels
df.is_train.value_counts()

(8124, 24)


True     6499
False    1625
Name: is_train, dtype: int64

### Task 1. Which feature has the highest amount of unique values? (joint dataset)


In [6]:
most_diversive = {column:len(set(df[column].dropna())) for column in df.columns}

print(most_diversive)

{'bruises': 2, 'cap_color': 10, 'cap_shape': 6, 'cap_surface': 4, 'gill_attachment': 2, 'gill_color': 12, 'gill_size': 2, 'gill_spacing': 2, 'habitat': 7, 'odor': 9, 'population': 6, 'ring_number': 3, 'ring_type': 5, 'spore_print_color': 9, 'stalk_color_above_ring': 9, 'stalk_color_below_ring': 9, 'stalk_root': 5, 'stalk_shape': 2, 'stalk_surface_above_ring': 4, 'stalk_surface_below_ring': 4, 'target': 2, 'veil_color': 4, 'veil_type': 1, 'is_train': 2}


In [7]:
most_diversive = {column: df[column].value_counts(normalize = True) for column in df.columns}
most_diversive

{'bruises': no         0.584441
 bruises    0.415559
 Name: bruises, dtype: float64, 'cap_color': brown       0.281142
 gray        0.226489
 red         0.184638
 yellow      0.131955
 white       0.128016
 buff        0.020679
 pink        0.017725
 cinnamon    0.005416
 green       0.001969
 purple      0.001969
 Name: cap_color, dtype: float64, 'cap_shape': convex     0.450025
 flat       0.387986
 knobbed    0.101920
 bell       0.055638
 sunken     0.003939
 conical    0.000492
 Name: cap_shape, dtype: float64, 'cap_surface': scaly      0.399311
 smooth     0.314623
 fibrous    0.285574
 grooves    0.000492
 Name: cap_surface, dtype: float64, 'gill_attachment': free        0.974151
 attached    0.025849
 Name: gill_attachment, dtype: float64, 'gill_color': buff         0.212703
 pink         0.183653
 white        0.147957
 brown        0.129000
 gray         0.092565
 chocolate    0.090103
 purple       0.060561
 black        0.050222
 red          0.011817
 yellow       0.01058

In [8]:
df_train['target'].value_counts(normalize = True)

1    0.51331
0    0.48669
Name: target, dtype: float64

'gill_color': 12

### Task 2
**As a preparation, one would spend up to 15-30 minutes on exploratory data analysis (EDA)** - make sure you understand how features are distributed in train/test, what they look like, are they ordinal/binary/categorical before moving further
<br>While doing it, please answer the questions

#### 2.1 Are there any features, obviously redundant to train on? If yes - what are they and why it's better to remove them?

In [9]:
print(df.shape)
# your code/hardcoded list goes here

# ---------------------------------------------------------------
redundant_columns = ['veil_type']
# ---------------------------------------------------------------
# lets drop these columns from joint dataset
df = df.drop(redundant_columns, axis=1, errors='ignore')
print(df.shape)

(8124, 24)
(8124, 23)


####  2.2 How many features (excluding target variable and train/test indexing columns) are:
- categorical (more than 2 unique values, no explicit ordering)
- ordinal (more than 2 unique values, explicit ordering)
- binary (2 unique values, doesn't matter whether it has ordering or is "yes/no" styled) 

In [10]:
# your code goes here
# ---------------------------------------------------------------
ordinal_cols = sorted(['ring_number'])
binary_cols = sorted([ column  for column in df_test.columns if len(set(df_test[column].dropna())) == 2])
categorical_cols = sorted(['cap_color','cap_shape','cap_surface','gill_color','habitat','odor','population','spore_print_color',
                         'ring_type','stalk_color_above_ring','stalk_color_below_ring','stalk_root',
                          'stalk_surface_above_ring','veil_color','stalk_surface_below_ring'])
# ---------------------------------------------------------------
print('categorical: {}\nordinal: {}\nbinary: {}'.format(
    len(categorical_cols), len(ordinal_cols), len(binary_cols)))

categorical: 15
ordinal: 1
binary: 5


In [11]:
# To be used in training, data must be properly encoded
from collections import defaultdict

# function to encode categorical data


def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat([ df[cat_cols] for df in df_list],axis=0).fillna('').apply(lambda x: d[x.name].fit(x))
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(lambda x: d[x.name].transform(x))


# label encode data (categorical + binary)
__encode_categorical(df_list=[df], cat_cols=categorical_cols+binary_cols)
# make sure you encode the only ordinal column in correct order
df[ordinal_cols[0]] = df[ordinal_cols[0]].map({'none': 0, 'one': 1, 'two': 2})

# define useful feature columns to be used for training
# (union of all columns discussed above)
columns_to_use = ordinal_cols + binary_cols + categorical_cols

### Task 3. Prepare cross-validation strategy and perform comparison of 2 baseline models (linear vs tree-based)

### =====================================================
#### Briefly about Validation / Cross-Validation

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but **would fail to predict anything useful on yet-unseen data. This situation is called overfitting**. 
<br>To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set ```X_test, y_test```. 
<br>Note that the word “experiment” is not intended to denote academic use only, because even in commercial settings machine learning usually starts out experimentally.

When evaluating different settings (“hyperparameters”) for estimators, **there is still a risk of overfitting on the test set** because the parameters can be tweaked until the estimator performs optimally. 
<br>This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. 
<br>To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, **we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.**

A solution to this problem is a procedure called **cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV**. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

- A model is trained using k-1 of the folds as training data;
- the resulting model is validated on the remaining part of the data 
<br>(i.e., it is used as a test set to compute a performance measure such as accuracy).
        
<img src="https://hsto.org/files/b1d/706/e6c/b1d706e6c9df49c297b6152878a2d03f.png"/ style="width:75%">

The performance measure reported by k-fold cross-validation **is then the average of the values computed in the loop**. 
<br>This approach can be computationally expensive, but does not waste too much data (as it is the case when fixing an arbitrary test set), which is a major advantage in problem such as inverse inference where the number of samples is very small.


Some classification problems can **exhibit a large imbalance in the distribution of the target classes: for instance there could be several times more negative samples than positive samples**. 
<br>In such cases it is recommended to use **stratified sampling** as implemented in sklearn's StratifiedKFold and StratifiedShuffleSplit to ensure that relative class frequencies is approximately preserved in each train and validation fold.

More details about different cross-validation strategies, implemented in sklearn, can be found [here](http://scikit-learn.org/stable/modules/cross_validation.html)
### =====================================================

Prepare KFold with 5 splits, stratified by target variable, shuffled, with fixed random_state = 42
<br>**Don't forget to filter by column 'is_train'!**
<br>Fit models on subset of features: [columns_to_use]

In [12]:
from os import cpu_count

n_jobs = max(cpu_count()-1, 1)

# cross-validation iterator
kf = StratifiedKFold(n_splits=5,shuffle = True, random_state=42)
# xtrain, ytrain, DataFrame-like
xtrain = df[columns_to_use].loc[df['is_train']==True]
ytrain =  df['target'].loc[df['is_train']==True]

In [13]:
# create Logistic Regression with default params, random_state=42
lr = LogisticRegression(random_state=42)

# estimate its f1-score with cross-validation
scores_lr = cross_val_score(
    estimator=lr,
    X=xtrain, 
    y=ytrain, 
    scoring='f1',
    cv=kf,
    n_jobs=n_jobs).mean()
print('LR scoring: {:.2f}'.format(scores_lr))

LR scoring: 0.89


In [14]:
# create Decision Tree with default params, max_depth=4, random_state=42
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
# estimate its f1-score with cross-validation (cross_val_score)
scores_dt = cross_val_score(
    estimator=dt,
    X=xtrain, 
    y=ytrain, 
    scoring='f1',
    cv=kf, 
    n_jobs=n_jobs).mean()
print('DT scoring: {:.2f}'.format(scores_dt))
# ---------------------------------------------------------------

DT scoring: 0.90


Why is a score of Linear Regression lower than correspondent one of DT?
1. Is everything OK with the data format for linear models? (revision of 2 previous lectures). 
1. If not, what else you should do to use the data appropriately for linear models?
1. Why didn't point 1. affect Decision Tree performance?

### Task 4. Now it's time to do some hyperparam tuning
Perform suitable hyperparam tuning using created above cross-validation strategy
<br>Main parameters to perform grid-search for:
- max_depth (1,2,...None)
- min_samples_leaf (1,2,...)
- criterion (gini, entropy)
- weight (none, balanced)
- max_features (sqrt(features), 50%, 75%, all of them, ...)
- other params available, see documentation

So - use your fantasy for filling-in abovementioned lists

You should receive **a gain of 0.01 in f1-score or higher**
<br>(current benchmark = +0.0268 gain)

In [21]:
%%time
# create base model (DT, random state = 42)
estimator = DecisionTreeClassifier(random_state = 42)

# create parameter grid
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
params = {'max_depth':[3,4,5,6,7],'min_samples_leaf':[2,3,5,7], 'criterion':['gini','entropy'],
          'class_weight':['balanced'],'max_features':['sqrt','log2',0.75,0.5,0.4],
         'min_samples_split':[2,3,4],'presort':[True,False]}

# create grid search object
gs = GridSearchCV(
    estimator=estimator,  # base model
    param_grid=params,  # params grid to search within
    cv=kf,  # cross-validation strategy
    error_score=1,  # warnings only
    scoring='f1',  # f1-score
    # thread count, the higher count - the faster
    n_jobs=n_jobs)

# perform grid search on TRAIN dataset ('is_train' filtering)
gs.fit(X=xtrain, y=ytrain)
# -------------------------------------------------------------
# extract best score on cross-validation
best_score = gs.best_score_
# extract the estimator (DT) with best params on cross-validation
best_dt = gs.best_estimator_
# check gain in f1-score
print('f1-score best: {:.4f}, +{:.4f} better than baseline'.format(
  best_score, (best_score - scores_dt))
)

f1-score best: 0.9282, +0.0269 better than baseline
CPU times: user 11.6 s, sys: 253 ms, total: 11.8 s
Wall time: 1min 5s


In [22]:
# take a look on the best model, compare with the baseline
best_dt

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=7, max_features=0.75, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=True, random_state=42,
            splitter='best')

In [23]:
# check performance on holdout dataset, unseen before (filter 'is_train' == False)
# appropriate df_test data subset from 'df' dataframe
xtest = df[columns_to_use].loc[df['is_train']==False]
# fit baseline model 'dt' on xtrain, ytrain (because it's not fitted yet)
dt = DecisionTreeClassifier(max_depth=3,random_state = 42)
dt.fit(xtrain, ytrain)

# baseline model
y_true = pd.read_csv('4-mushrooms-y_test.csv')

In [24]:
y_pred_baseline = dt.predict(xtest)

print('Base on train:   {:.4f}\nBase on holdout: {:.4f}\ndiff: {:.4f}'.format(
    scores_dt, 
    f1_score(y_true, y_pred_baseline),
    scores_dt - f1_score(y_true, y_pred_baseline)))

# best model
y_pred_best = best_dt.predict(xtest)

print('\nBest on train:   {:.4f}\nBest on holdout: {:.4f}\ndiff: {:.4f}'.format(
    best_score, 
    f1_score(y_true, y_pred_best),
    best_score - f1_score(y_true, y_pred_best)
))

Base on train:   0.9012
Base on holdout: 0.8888
diff: 0.0125

Best on train:   0.9282
Best on holdout: 0.9194
diff: 0.0088


Now you can see that 
<br>**absolute values of f1-score is higher and distance between train|holdout is lower** <br>for **best model** in comparison to **baseline**

**Bonus question**:

Consider two possibilities:
- (a) you have trained **one best** (on cross-validation) Decision Tree
- (b) you randomly choose 25 subsets of 70% of training data, fits "overfitted" (max_depth=None) Decision Trees on it - each of them performs slightly worse than Tree in (a), and then average predicted results over all 25 models (overfitted trees)

**Which one of them would most likely give the best results on hold-out dataset? What makes you think that way?**

In [50]:
rf = RandomForestClassifier(n_estimators = 25, random_state = 42)
rf.fit(xtrain,ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [51]:
y_pr = rf.predict(xtest)

In [52]:
print( f1_score(y_true, y_pr))

0.9168674698795181


In [53]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)